In [1]:
# import the necessary packages
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import imutils
import pickle
import time
import cv2
import os, sys

# Import functions from gunController utilities file
sys.path.append('../utilities')
from gunController import *
from ROIClass import *
from alignFaces import *

In [2]:
data_params = {
    "detector":"../../facial_detection/models/", # Path to OpenCV's deep learning face detector
    "embedding_model":"../models/openface.nn4.small2.v1.t7", # Path to OpenCV's embedding model
    "recognizer":"../../output/recognizer.pickle", # Path to model trained to recognize faces
    "label_encoder":"../../output/le.pickle", # Path to label encoder
    "detection_confidence":0.5, # Min probability to filter weak detections
    "recognition_confidence":0.5 # Min probability to filter weak recognitions
}

In [3]:
# gc = gunController()

In [4]:
# The below function is responsible for the firing of the nerf gun turret
def turret_controller(ROI_list, center):
    # If there is at least face in the ROI_list...
    if len(ROI_list):
        # The list is sorted based on the ROI's distance from the center of the frame.
        ROI_list.sort()
        
        # The ROI closest to center is the first item of the list. This is known as the current ROI
        curr_ROI_obj = ROI_list[0]
        dist_from_center = curr_ROI_obj.dist_from_center
        
        # If the current ROI is 0, then the face is aligned with the center of the frame 
        if not dist_from_center:
            print("Pull trigger")
            # gc.pull_trigger()
        else:
            print("Release trigger")
            # gc.release_trigger()
            if center < curr_ROI_obj.startX:
                print("Move left")
                # gc.move_left()
            else:
                print("Move right")
                # gc.move_right()
    else:
        print("Release trigger")
        # gc.release_trigger()

In [5]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = os.path.sep.join([data_params["detector"], "deploy.prototxt"])
modelPath = os.path.sep.join([data_params["detector"], "res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

[INFO] loading face detector...


In [6]:
# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(data_params["embedding_model"])

[INFO] loading face recognizer...


In [7]:
# load the actual face recognition model along with the label encoder
recognizer = pickle.loads(open(data_params["recognizer"], "rb").read())
le = pickle.loads(open(data_params["label_encoder"], "rb").read())

In [8]:
# initialize the video stream, then allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

# start the FPS throughput estimator
fps = FPS().start()

# initialize a "friendly" variable
friendly = False

# loop over frames from the video file stream
while True:
    # grab the frame from the threaded video stream
    frame = vs.read()
    
    # A new list containing the faces of the current frame is instantiated
    ROI_list = []
    
    # resize the frame to have a width of 600 pixels (while
    # maintaining the aspect ratio), and then grab the image
    # dimensions
    frame = imutils.resize(frame, width=600)
    
    (h, w) = frame.shape[:2]
    center = w // 2
    
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)
    
    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the prediction
        confidence = detections[0, 0, i, 2]
        
        # filter out weak detections
        if confidence > data_params["detection_confidence"]:
            # compute the (x, y)-coordinates of the bounding box for
            # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            # extract the face ROI
            face = frame[startY:endY, startX:endX]
            
            (fH, fW) = face.shape[:2]
            
            # ensure the face width and height are sufficiently large
            if fW < 10 or fH < 10:
                continue
                
            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()
            
            # perform classification to recognize the face
            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            
            # filter out weak recognitions
            if proba > data_params["recognition_confidence"]:
                name = le.classes_[j]
            else:
                name = "unknown"
            
            # check if the recognized face is friendly or not
            if name == "unknown":
                friendly = False
                # The coords and centers are used as constructor parameters to the ROIFace class
                ROI_list.append(ROIFace(startX, startY, endX, endY, center))
            else:
                friendly = True
            
            # draw the bounding box of the face along with the
            # associated probability, the name label, and friendly label
            text = "{}: {:.2f}%".format(name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    
    # The ROI list of the current frame is examined
    turret_controller(ROI_list, center)
                        
    # update the FPS counter
    fps.update()
    
    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
        
# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] starting video stream...
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release 

Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
Release trigger
[INFO] elasped time: 16.67
[INFO] approx. FPS: 31.62
